In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joelma.fatima\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joelma.fatima\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import math

class Coletor:
    def __init__(self):
        self.urls = []

    def adicionar_url(self, url):
        self.urls.append(url)

    def extrair_texto(self, html):
        soup = BeautifulSoup(html, 'html.parser')
        texto = soup.get_text(separator=' ')
        return texto

    def coletar(self):
        textos = {}
        for url in self.urls:
            response = requests.get(url)
            if response.status_code == 200:
                html = response.text
                texto = self.extrair_texto(html)
                textos[url] = texto
        return textos

class Indexador:
    def __init__(self):
        self.indice_invertido = defaultdict(list)
        self.stopwords = set(stopwords.words('portuguese'))
        self.stopwords.update(string.punctuation)

    def indexar(self, textos):
        for url, texto in textos.items():
            tokens = word_tokenize(texto.lower())
            for token in tokens:
                if token not in self.stopwords:
                    self.indice_invertido[token].append(url)

    def calcular_tfidf(self, num_documentos):
        tfidf = {}
        for termo, ocorrencias in self.indice_invertido.items():
            idf = math.log(num_documentos / len(ocorrencias))
            tfidf[termo] = idf
        return tfidf

class Buscador:
    def __init__(self, indice_invertido, tfidf):
        self.indice_invertido = indice_invertido
        self.tfidf = tfidf

    def buscar(self, consulta):
        tokens_consulta = word_tokenize(consulta.lower())
        scores = defaultdict(float)
        for token in tokens_consulta:
            if token in self.indice_invertido:
                idf = self.tfidf.get(token, 0)
                for documento in self.indice_invertido[token]:
                    scores[documento] += idf
        resultados = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        return resultados

class Avaliador:
    def __init__(self, relevantes):
        self.relevantes = relevantes

    def avaliar(self, resultados):
        relevantes_retrieved = 0
        relevantes_totais = len(self.relevantes)
        for url, _ in resultados:
            if url in self.relevantes:
                relevantes_retrieved += 1
        precisao = relevantes_retrieved / len(resultados) if len(resultados) > 0 else 0
        revocacao = relevantes_retrieved / relevantes_totais
        return precisao, revocacao

# Exemplo de uso
if __name__ == "__main__":
    urls_coleta = [
        "https://www.gov.br/inep/pt-br/areas-de-atuacao/avaliacao-e-exames-educacionais/enem",
        "https://ouropreto.ifmg.edu.br/ouropreto",
        "https://brasilescola.uol.com.br"
    ]
    consultas = ["educação", "enem", "educação e enem", "educação ou enem"]

    coletor = Coletor()
    for url in urls_coleta:
        coletor.adicionar_url(url)
    textos_coletados = coletor.coletar()

    indexador = Indexador()
    indexador.indexar(textos_coletados)
    tfidf = indexador.calcular_tfidf(len(textos_coletados))

    buscador = Buscador(indexador.indice_invertido, tfidf)

    relevantes = ["https://www.gov.br/inep/pt-br/areas-de-atuacao/avaliacao-e-exames-educacionais/enem"]
    avaliador = Avaliador(relevantes)

    for consulta in consultas:
        resultados = buscador.buscar(consulta)
        precisao, revocacao = avaliador.avaliar(resultados)
        print(f"Consulta: {consulta}")
        print("Resultados:")
        for url, score in resultados:
            print(f"{url}: {score}")
        print(f"Precision: {precisao}, Recall: {revocacao}")
        print()


Consulta: educação
Resultados:
https://ouropreto.ifmg.edu.br/ouropreto: -3.0757749812275272
https://brasilescola.uol.com.br: -24.60619984982022
https://www.gov.br/inep/pt-br/areas-de-atuacao/avaliacao-e-exames-educacionais/enem: -172.24339894874163
Precision: 0.3333333333333333, Recall: 1.0

Consulta: enem
Resultados:
https://ouropreto.ifmg.edu.br/ouropreto: -5.957850310475219
https://www.gov.br/inep/pt-br/areas-de-atuacao/avaliacao-e-exames-educacionais/enem: -62.55742825998983
https://brasilescola.uol.com.br: -107.2413055885539
Precision: 0.3333333333333333, Recall: 1.0

Consulta: educação e enem
Resultados:
https://ouropreto.ifmg.edu.br/ouropreto: -9.033625291702746
https://brasilescola.uol.com.br: -131.8475054383741
https://www.gov.br/inep/pt-br/areas-de-atuacao/avaliacao-e-exames-educacionais/enem: -234.80082720873162
Precision: 0.3333333333333333, Recall: 1.0

Consulta: educação ou enem
Resultados:
https://ouropreto.ifmg.edu.br/ouropreto: -9.033625291702746
https://brasilescola.u

Utilizando  o modelo TF-IFF para o processamento de consultas e para ranqueamento de resultados. 
Implemnetação da avaliação das consultas usando métricas de precisão e revocação. 

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class IndexadorTFIDF:
    def __init__(self):
        self.documentos = []
        self.tf_idf_model = None
        self.stopwords = set(stopwords.words('portuguese'))

    def indexar(self, documentos):
        self.documentos = documentos
        corpus = [' '.join([token for token in word_tokenize(doc['texto'].lower()) if token not in self.stopwords]) for doc in documentos]
        self.tf_idf_model = TfidfVectorizer()
        self.tf_idf_model.fit(corpus)

    def processar_consulta(self, consulta):
        consulta = ' '.join([token for token in word_tokenize(consulta.lower()) if token not in self.stopwords])
        consulta_tf_idf = self.tf_idf_model.transform([consulta])
        scores = cosine_similarity(consulta_tf_idf, self.tf_idf_model.transform([doc['texto'] for doc in self.documentos]))[0]
        resultados = sorted(list(enumerate(scores)), key=lambda x: x[1], reverse=True)
        return resultados

# Exemplo de uso
if __name__ == "__main__":
    # Exemplo de documentos e consulta
    documentos = [
        {'texto': 'Este é um exemplo de documento sobre análise de sentimentos.'},
        {'texto': 'Neste documento, falamos sobre processamento de linguagem natural e indexação.'},
        {'texto': 'A análise de sentimentos é uma área importante da mineração de textos.'}
    ]
    consulta = 'análise de sentimentos e processamento de linguagem natural'

    # Indexar documentos
    indexador = IndexadorTFIDF()
    indexador.indexar(documentos)

    # Processar consulta
    resultados = indexador.processar_consulta(consulta)

    print("Resultados da consulta:")
    for idx, score in resultados:
        print(f"Documento {idx}: {score}")


Resultados da consulta:
Documento 1: 0.5500248858857211
Documento 0: 0.3116940163436369
Documento 2: 0.24985512660870635


Com esses documentos e consulta, o resultado da consulta deve priorizar o segundo documento, que contém mais termos relevantes para a consulta.

In [8]:
# Exemplo de documentos e consulta
documentos = [
    {'texto': 'Este é um exemplo de documento sobre análise de sentimentos.'},
    {'texto': 'Neste documento, falamos sobre processamento de linguagem natural e indexação.'},
    {'texto': 'A análise de sentimentos é uma área importante da mineração de textos.'}
]

consulta = 'processamento de linguagem natural e mineração de textos'


este caso, o resultado da consulta deve priorizar o segundo e terceiro documentos, pois eles contêm termos relevantes para a consulta.

In [9]:
# Exemplo de documentos e consulta
documentos = [
    {'texto': 'Este é um exemplo de documento sobre aprendizado de máquina.'},
    {'texto': 'Aqui discutimos sobre redes neurais e algoritmos de aprendizado.'},
    {'texto': 'A aprendizagem profunda é uma técnica avançada de aprendizado de máquina.'}
]

consulta = 'aprendizado de máquina e redes neurais'


In [10]:
# Exemplo de uso
if __name__ == "__main__":
    # Exemplo de documentos e consulta
    documentos = [
        {'texto': 'Este é um exemplo de documento sobre análise de sentimentos.'},
        {'texto': 'Neste documento, falamos sobre processamento de linguagem natural e indexação.'},
        {'texto': 'A análise de sentimentos é uma área importante da mineração de textos.'}
    ]
    consulta = 'análise de sentimentos e processamento de linguagem natural'

    # Indexar documentos
    indexador = IndexadorTFIDF()
    indexador.indexar(documentos)

    # Processar consulta
    resultados = indexador.processar_consulta(consulta)

    print("Resultados da consulta:")
    for idx, score in resultados:
        print(f"Documento {idx}: {score}")


Resultados da consulta:
Documento 1: 0.5500248858857211
Documento 0: 0.3116940163436369
Documento 2: 0.24985512660870635


In [13]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Baixar recursos necessários do NLTK (execute apenas uma vez)
nltk.download('vader_lexicon')

# Inicializar o analisador de sentimentos
sia = SentimentIntensityAnalyzer()

# Função para interpretar o score de sentimento
def interpretar_sentimento(score):
    if score['compound'] >= 0.05:
        return "positivo"
    elif score['compound'] <= -0.05:
        return "negativo"
    else:
        return "neutro"

# Frases de exemplo
frases = [
    "Eu amo esse filme! É incrível.",
    "Que dia terrível... Não consigo lidar com isso.",
    "O tempo está bom lá fora.",
    "Isso é tão legal!",
    "Não gosto desse produto. Foi uma compra ruim."
]

# Analisar o sentimento de cada frase
for frase in frases:
    score = sia.polarity_scores(frase)
    sentimento = interpretar_sentimento(score)
    print(f"Frase: '{frase}' - Sentimento: {sentimento}")


Frase: 'Eu amo esse filme! É incrível.' - Sentimento: neutro
Frase: 'Que dia terrível... Não consigo lidar com isso.' - Sentimento: neutro
Frase: 'O tempo está bom lá fora.' - Sentimento: neutro
Frase: 'Isso é tão legal!' - Sentimento: positivo
Frase: 'Não gosto desse produto. Foi uma compra ruim.' - Sentimento: neutro


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\joelma.fatima\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
